# Algoritimo de Leitura

In [1]:
# imports
import pdfplumber
import os
from pdf2image import convert_from_path
import pytesseract
pytesseract.pytesseract.tesseract_cmd=r'Tesseract-OCR\\tesseract.exe'
import cv2

In [40]:
# Metódo para lista os itens do diretório de leitura
def aqr_dir(url):
    dir_ = url
    path = [os.path.join(dir_, nome) for nome in os.listdir(dir_)]
    url_res = [arq for arq in path if os.path.isfile(arq)]
    arq = [nome.replace('.pdf',"") for nome in os.listdir(dir_)]
    
    return url_res, arq


# Metódo para realizar leitura de arquivo de origem pdf
def pdf(url):
    with pdfplumber.open(url) as pdf:        
        page =[pdf.pages[i] for i in range(len(pdf.pages))]      
        extracText = [page[i].extract_text() for i in range(len(page))]
                  
        return extracText
    

# Metódo para realizar leitura de arquivos de formatos img ou pdf scaneados
def ocr(url, arq):
        
    text = []
    res = []        
            
    def reduce(img):  
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        th, threshed = cv2.threshold(gray, 250, 255, cv2.THRESH_BINARY_INV)
        kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (80,80))
        morphed = cv2.morphologyEx(threshed, cv2.MORPH_CLOSE, kernel)
        cnts = cv2.findContours(morphed, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)[-2]
        cnt = sorted(cnts, key=cv2.contourArea)[-1]
        x,y,w,h = cv2.boundingRect(cnt)
        dst = img[y:y+h, x:x+w]
        return dst
    
    img = convert_from_path(url, dpi=300, poppler_path=r'poppler-0.68.0\\bin')   
    im = [imgs.save('nf_img\\'+str(arq+'.png'),"PNG") for imgs in img]
                
    nfdir = 'nf_img'
    path = [os.path.join(nfdir, nome) for nome in os.listdir(nfdir)]
    url = [arq for arq in path if os.path.isfile(arq)]
    url_img = [u for u in url if ".png" in u]
                
    imagem = cv2.imread(url_img[0])
    img_res = reduce(imagem)
                
    gray = cv2.cvtColor(img_res, cv2.COLOR_BGR2GRAY)
    adp = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,85,80)        
    tx = (pytesseract.image_to_string(adp, lang='por',config='--psm 6'))
                
    [os.remove(u) for u in url_img if ".png" in u]
                
    return tx   

In [41]:
# Chamada ao metódo de leitura de diretórios
url, name = aqr_dir('nf')

In [42]:
# Leitura dos itens
tx = [p if p!= None else ocr(url[i],name[i]) for i in range(len(url)) for p in pdf(url[i])]

In [43]:
tx

['Vu PREFEITURA DA CIDADE DO RIO DEJANEIRO — |É |Memen SA noa\nM ” DOOOO033\n= " SECRETARIA MUNICIPAL DE FAZENDA Data e Hora de Emissão\nç Z A D4/02/2021 09:34:33\n% \' NOTA FISCAL DE SERVIÇOS ELETRONICA - NFS-e d o\noA TS o ódigo de Verificação\n2O210217U34126214000120 - NOTA CARIOCA - RH2B-ENJT\nPRESTADOR DE SERVIÇOS\nCPF/CNPJ: 34.126.214/0001-20 Inscrição Municipal: 1.185.508-3 Inscrição Estadual: =--\nNome/Razão Social: AZ 91 REPRESENTACOES LTDA\nNome Fantasia: AZ 91 REPRESENTAÇÕES LTDA Tel.: 21-025973639\nEndereço: RUA DEODATO DE MORAES 76, COB 02 - BARRA DA TIJUCA - CEP: 22620-070\nMunicípio: RIO DE JANEIRO UF: RJ E-mail: ==---\nTOMADOR DE SERVIÇOS\nCPF/CNPJ: 47.173.950/0001-81 Inscrição Municipal: =-=-- Inscrição Estadual: --—--\nNome/Razão Social: RINNAI BRASIL TEC DE AQUECIMENTO LTDA.\nEndereço: RUA TTE ONOFRE RODRIGUES DE AGUIAR 200 - VILA INDUSTRIAL Tel : ===\n* CEP: 08770-041 .\nMunicípio: MOGI DAS CRUZES UF: SP E-mail: pedro&rinnal.com.br\nDISCRIMINAÇÃO DOS SERVIÇOS\nServi